In [1]:
import bayesfast as bf
import numpy as np

In [2]:
from scipy.stats import special_ortho_group

D = 32 # number of dims
Q = 0.01
lower = np.full(D, -15.) # lower bound of the prior
upper = np.full(D, 15.) # upper bound of the prior
bound = np.array((lower, upper)).T
diff = bound[:, 1] - bound[:, 0]
const = np.sum(np.log(diff)) # normalization of the flat prior

np.random.seed(0)
A = special_ortho_group.rvs(D) # random rotation of the bananas

def logp(x):
    x = x @ A.T
    return -np.sum((x[..., ::2]**2 - x[..., 1::2])**2 / Q + 
                   (x[..., ::2] - 1)**2, axis=-1) - const

def grad(x):
    x = x @ A.T
    _pfpx2i1 = 2 * (x[..., 1::2] - x[..., ::2]**2) / Q
    _pfpx2i = 2 * (x[..., ::2] - 1) - 2 * x[..., ::2] * _pfpx2i1
    res = np.empty_like(x)
    res[..., ::2] = _pfpx2i
    res[..., 1::2] = _pfpx2i1
    return -res @ A

In [3]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:44755 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 126.70 GB


In [4]:
den = bf.DensityLite(logp=logp, grad=grad, input_size=D, input_scales=bound,
                     hard_bounds=True)
sample_trace = {'n_chain': 8, 'n_iter': 2500, 'n_warmup': 1000}
rec = bf.Recipe(density=den, sample={'sample_trace': sample_trace,
                'random_state': 0}, post={}, client=client)

In [5]:
rec.run()

/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #6 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #4 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #4 : invalid value encountered in true_divide
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #3 : divide by zero encountered in log
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #3 : invalid value encountered in true_divide
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: RuntimeWarning:  CHAIN #3 : divide by zero encountered in true_divide
  warnings.warn(msg[1], msg[0])
/global/u2/h/hejia/bayesfast/bayesfast/core/sample.py:104: R

 CHAIN #2 : sampling proceeding [ 500 / 2500 ], last 500 samples used 43.41 seconds, while divergence encountered in 27 sample(s). (warmup)
 CHAIN #1 : sampling proceeding [ 500 / 2500 ], last 500 samples used 46.72 seconds. (warmup)
 CHAIN #4 : sampling proceeding [ 500 / 2500 ], last 500 samples used 51.88 seconds, while divergence encountered in 30 sample(s). (warmup)
 CHAIN #0 : sampling proceeding [ 500 / 2500 ], last 500 samples used 53.49 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 500 / 2500 ], last 500 samples used 55.15 seconds, while divergence encountered in 42 sample(s). (warmup)
 CHAIN #6 : sampling proceeding [ 500 / 2500 ], last 500 samples used 59.42 seconds, while divergence encountered in 50 sample(s). (warmup)
 CHAIN #5 : sampling proceeding [ 500 / 2500 ], last 500 samples used 59.96 seconds, while divergence encountered in 29 sample(s). (warmup)
 CHAIN #7 : sampling proceeding [ 500 / 2500 ], last 500 samples used 65.94 seconds, while divergence encountere

/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/glo

 ***** PostStep finished. ***** 



In [6]:
rec.get().logz, rec.get().logz_err # fiducial value: logz = -127.364

(-127.33558361243203, 0.04669610491992853)